# Data cleaning

### This notebook goes through the datacleaning process

In [548]:
# imports
import pandas as pd
import numpy as np

In [549]:
# reading in the data frames
df = pd.read_csv('item_desc_6.csv', index_col=[0])

In [550]:
df.head()

,username,sold_price,designer,category,description,sub_title,image_count,size_color_cond,feedback_count,link
0,jollytyler,$285 (Sold Price),Soulive,Soulive Light Jackets,Features an allover indigo bandana patchwork d...,Patchwork Bandana Mercury Jacket Visvim ICT,14,Size: US L / EU 52-54 / 3 Color: Indigo Condi...,268 Feedback,https://www.grailed.com/listings/19144426-soul...
1,eWarbs,$135 (Sold Price),Soulive,Soulive Light Jackets,Amazing patchwork denim jacket with western bu...,Soulive Denim Kimono Jacket,12,Size: US L / EU 52-54 / 3 Color: Blue Conditi...,19 Feedback,https://www.grailed.com/listings/18632785-soul...
2,RX7,$310 (Sold Price),Soulive,Soulive Light Jackets,Great condition\nNo flaws\nBlanket lined to ke...,Kimono jacket tan brown navy blue made japan c...,4,Size: US L / EU 52-54 / 3 Color: Brown Condit...,470 Feedback,https://www.grailed.com/listings/17749036-soul...
3,ShinjiNakamura,$259 (Sold Price),Soulive,Soulive Denim Jackets,BLX14-01614\nA model that updated the SOULIVE ...,Soulive Western Haori indigo - M/L - New,5,Size: US L / EU 52-54 / 3,NaN,https://www.grailed.com/listings/15026333-soul...
4,lilspookymane,$85 (Sold Price),Blue Owl x Soulive,Blue Owl Casual Pants,Size 2. Great condition. Worn less than 5 time...,Sashiko Hakama Pants - Indigo,4,Size: US 32 / EU 48 Color: Indigo Condition: ...,66 Feedback,https://www.grailed.com/listings/15757935-blue...


In [551]:
# checking for nans and empty strings
df.isnull().sum()

username            49
sold_price          33
designer            33
category            33
description         34
sub_title           33
image_count          0
size_color_cond     33
feedback_count     572
link                 0
dtype: int64

In [552]:
# checking datatypes
df.dtypes

username           object
sold_price         object
designer           object
category           object
description        object
sub_title          object
image_count         int64
size_color_cond    object
feedback_count     object
link               object
dtype: object

## Cleaning up the feedback_count column

1. making column a float
2. replacing nan with 0

In [553]:
df['feedback_count'] = df['feedback_count'].str.extract('(\d+)').astype(float)

In [554]:
# changing missing feedback_count to zero instead of an empty string
df['feedback_count'] = df['feedback_count'].fillna(0)

In [555]:
df['feedback_count']

0        268.0
1         19.0
2        470.0
3          0.0
4         66.0
         ...  
19994     46.0
19995    109.0
19996      3.0
19997    204.0
19998     14.0
Name: feedback_count, Length: 19999, dtype: float64

## Dropping Remaining NANs

decided that there wasnt much of a way to fix these values and there were so few overall.

In [556]:
df.dropna(inplace=True)

In [557]:
df.isnull().sum()

username           0
sold_price         0
designer           0
category           0
description        0
sub_title          0
image_count        0
size_color_cond    0
feedback_count     0
link               0
dtype: int64

## Cleaning up sold_price column

1. cast column as a float
2. check for an zeros or negative numbers to remove

In [558]:
# changing sold_price to a float
df['sold_price'] = df['sold_price'].str.extract('(\d+)').astype(float)
df['sold_price']

0        285.0
1        135.0
2        310.0
3        259.0
4         85.0
         ...  
19994    235.0
19995    175.0
19996    140.0
19997    230.0
19998     59.0
Name: sold_price, Length: 19949, dtype: float64

In [559]:
# checking for sold_price less than or equal to zero
df[df['sold_price'] <= 0]
# no values to remove

,username,sold_price,designer,category,description,sub_title,image_count,size_color_cond,feedback_count,link


## Cleaing up category column

1. removing designer name from category
2. drop rows if category contains less than 10

In [560]:
# reseting the index given it had values dropped
df.index = range(len(df))

In [561]:
# making sure category is a string
df['category'] = df['category'].astype(str)

In [562]:
# making sure designer is a string
df['designer'] = df['designer'].astype(str)

In [563]:
# function to replace designers in category column with empty string leaving only the item category
def replace_designer(row):
    words = df['designer'][row].split() # splitting the designer into multple words incase multiple designers are present
    for word in words:
        if word in df['category'][row]:
            df['category'][row] = df['category'][row].replace(word,'') # goes through each word and replaces with empty string if present
    return df['category'][row].strip()

In [564]:
# using function on all data to remove designer from category
for i in range(len(df['category'])):
    replace_designer(i)

<ipython-input-563-8bd2803f4eaa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'][row] = df['category'][row].replace(word,'') # goes through each word and replaces with empty string if present


In [565]:
# dropping all rows if category count is less than 10
df = df.groupby('category').filter(lambda x : len(x)>10)

## Cleaning up size_color_condition column

1. create a column for size
2. create a column for color
3. create a column for condition
4. drop all rows that all 3 columns could not be created

In [566]:
# resetting the index
df.index = range(len(df))

In [568]:
# extracting the size into a new size column
def get_size(row):
    words = df['size_color_cond'][row]
    try:
        df['size'][row] = words[words.find('Size:'):words.find('Color:')].replace('Size:', '').strip() # will find all things after size
    except: 
        df['size'][row] = np.nan
    return df['size'][row]

In [569]:
# applying get_size to all rows
df['size'] = 0
for i in range(len(df['size_color_cond'])):
    get_size(i)

<ipython-input-568-3cca5f246ac1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'][row] = words[words.find('Size:'):words.find('Color:')].replace('Size:', '').strip()
C:\Users\frantzal\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [570]:
# extracting the color into a new color column
def get_color(row):
    words = df['size_color_cond'][row] # splitting the designer into multple words incase multiple designers are present
    try:
        df['color'][row] = words[words.find('Color:'):words.find('Condition:')].replace('Color:', '').strip()
    except:
        df['color'][row] = np.nan
    return df['color'][row]

In [571]:
# applying get_color to all rows
df['color'] = 'none'
for i in range(len(df['size_color_cond'])):
    get_color(i)

<ipython-input-570-beaff590a6ba>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['color'][row] = words[words.find('Color:'):words.find('Condition:')].replace('Color:', '').strip()


In [572]:
# extracting the condition into a new condition column
def get_condition(row):
    words = df['size_color_cond'][row] 
    try:
        df['condition'][row] = words[words.find('Condition:')::].replace('Condition:', '').strip()
    except:
        df['condition'][row] = np.nan
    return df['condition'][row]

In [573]:
# applying get_condition to all rows
df['condition'] = 'none'
for i in range(len(df['size_color_cond'])):
    get_condition(i)

<ipython-input-572-5710c0b43092>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['condition'][row] = words[words.find('Condition:')::].replace('Condition:', '').strip()


In [575]:
# replacing empty strings with nan
df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [576]:
# dropping rows that did not have all 3 categories, size color and condition
df.dropna(inplace=True)

## Cleaning up description column

1. remove new lines (\n) from description

In [578]:
# replacing \n with a space
df['description'] = df['description'].replace('\n',' ',regex=True)

## Cleaning up designer column

In [583]:
# resetting the index
df.index = range(len(df))

In [609]:
# function to change coloborations into one designer
def one_designer(row):
    word = df['designer'][row]
    words = word.split() 
    if 'x' in words:
        df['designer'][row] = df['designer'][row][:word.index('x')].strip() # if x is in the designer name will only return first designer
    return df['designer'][row]

In [611]:
# applyng one_designer to all rows
for i in range(len(df['designer'])):
    one_designer(i)

<ipython-input-609-0da5feab6913>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['designer'][row] = df['designer'][row][:word.index('x')].strip() # if x is in the designer name will only return first designer


## Final Dataframe

In [616]:
# dropping duplicates
df.drop_duplicates(inplace=True)

In [617]:
df

,username,sold_price,designer,category,description,sub_title,image_count,size_color_cond,feedback_count,link,size,color,condition
0,jollytyler,285.0,Soulive,Light Jackets,Features an allover indigo bandana patchwork d...,Patchwork Bandana Mercury Jacket Visvim ICT,14,Size: US L / EU 52-54 / 3 Color: Indigo Condi...,268.0,https://www.grailed.com/listings/19144426-soul...,US L / EU 52-54 / 3,Indigo,New
1,eWarbs,135.0,Soulive,Light Jackets,Amazing patchwork denim jacket with western bu...,Soulive Denim Kimono Jacket,12,Size: US L / EU 52-54 / 3 Color: Blue Conditi...,19.0,https://www.grailed.com/listings/18632785-soul...,US L / EU 52-54 / 3,Blue,Gently used
2,RX7,310.0,Soulive,Light Jackets,Great condition No flaws Blanket lined to keep...,Kimono jacket tan brown navy blue made japan c...,4,Size: US L / EU 52-54 / 3 Color: Brown Condit...,470.0,https://www.grailed.com/listings/17749036-soul...,US L / EU 52-54 / 3,Brown,Gently used
3,lilspookymane,85.0,Blue Owl,Casual Pants,Size 2. Great condition. Worn less than 5 time...,Sashiko Hakama Pants - Indigo,4,Size: US 32 / EU 48 Color: Indigo Condition: ...,66.0,https://www.grailed.com/listings/15757935-blue...,US 32 / EU 48,Indigo,Gently used
4,hsoandso,127.0,Soulive,Light Jackets,This Soulive Grand Haori jacket is like a vint...,Soulive Grand Haori jacket,6,Size: US M / EU 48-50 / 2 Color: Olive Condit...,1.0,https://www.grailed.com/listings/17286776-soul...,US M / EU 48-50 / 2,Olive,Gently used
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14821,Arbitrage,235.0,Oliver Peoples,Sunglasses,This Item Retailed for $510 Oliver Peoples Has...,O'Malley NYC OG Tortoise 48MM,9,Size: ONE SIZE Color: Tortoise Condition: New,46.0,https://www.grailed.com/listings/10797211-oliv...,ONE SIZE,Tortoise,New
14822,smittysan8970,175.0,The Row,Sunglasses,MSRP $510 9/10 condition - peep the photos Cle...,Oliver Peoples x The Row Board Meeting 2 Squar...,11,Size: ONE SIZE Color: Gold Condition: Gently ...,109.0,https://www.grailed.com/listings/10979242-the-...,ONE SIZE,Gold,Gently used
14823,LukasLukas,140.0,The Row,Casual Pants,"Beautiful off white The Row pants, super flowi...",The Row off white pants,4,Size: US 28 / EU 44 Color: Off white Conditio...,3.0,https://www.grailed.com/listings/10323649-the-...,US 28 / EU 44,Off white,Gently used
14824,fractors,230.0,The Row,Polos,"SS19 super fine black cotton long sleeve, exte...",Black Cotton Long Sleeve Polo,4,Size: US L / EU 52-54 / 3 Color: Black Condit...,204.0,https://www.grailed.com/listings/10289762-the-...,US L / EU 52-54 / 3,Black,New
